In [21]:
%%time
import numpy as np
import glob
import os
from pyuvdata import UVData, UVCal
from pyuvdata import utils as uvutils
from astropy import units as u
from astropy import constants as c
import pandas as pd
from astropy.time import Time
from astropy.coordinates import get_body, SkyCoord, AltAz, EarthLocation, Angle


# Calculate sun position
def SunAlt(times, location):
    sun = get_body('sun', times)
    sun_altaz = sun.transform_to(AltAz(location = location))
    return sun_altaz.alt.deg


JD = '2457552'
rawpath = '/lustre/aoc/projects/hera/plaplant/HERA19Golden/RawData/'+JD+'/'
outpath = '/lustre/aoc/projects/hera/jaguirre/HERA19Summer2020/RawData/'+JD+'/'

#load in all files for that day
filenames = sorted(glob.glob(os.path.join(rawpath, 'zen.' + JD +'*xx.HH.uvcRP')))

# There's no point in doing this multiple times
uvd = UVData()
uvd.read(filenames[0])
# HERA Lat / Lon
lat, lon, alt = uvd.telescope_location_lat_lon_alt_degrees
latitude = Angle(lat*u.deg)
longitude = Angle(lon*u.deg)
# This is GEODETIC
karoo = EarthLocation(longitude, latitude, height=alt*u.m)

data = {'Filename': [], 
        'Julian Date': [], 
        'LST (rad)': [], 
        'UTC': [],
        'LST (hr)': [],
        'Sun alt (deg)': []
       }

for filename in filenames:
    
    # creating output filename
    filename_only = filename.split('/')[-1]
    tmp = filename_only.split('.')
       
    #uvd = UVData()
    #uvd.read(filename)  # don't need to do this twice
    
    ntimes = uvd.Ntimes
    lst_tmp = np.unique(uvd.lst_array)
    time_tmp = np.unique(np.unique(uvd.time_array))
    times = Time(time_tmp, format='jd')
    
    """ This approach makes a list out of however many time samples are in a file (a variable
    number, otherwise we would just declare the size of the array at the outset)
    """
    data['Filename'] += [outfile for i in np.arange(ntimes)]
    data['Julian Date'] += list(time_tmp)  
    data['LST (rad)'] += list(lst_tmp)
    data['UTC'] += list(Time(time_tmp, format='jd').iso)
    data['LST (hr)'] += list(Angle(lst_tmp, u.rad).to_string(unit=u.hour))
    data['Sun alt (deg)'] += list(SunAlt(times, karoo))



dataframe = pd.DataFrame(data)
dataframe.to_csv(outpath+JD+'_summary.csv')
#dataframe.to_csv(JD+'_summary.csv')

CPU times: user 11.4 s, sys: 87.4 ms, total: 11.5 s
Wall time: 11.5 s
